In [ ]:
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
from matplotlib import colors
from math import sqrt
from math import log

# Move class for Breakthrough

In [ ]:
class Move(object):
    def __init__(self, color, x1, y1, x2, y2):
        self.color = color
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        
    def valid (self, board):
        if self.x2 >= Dx or self.y2 >= Dy or self.x2 < 0 or self.y2 < 0:
            return False
        if self.color == White:
            if self.x2 != self.x1 + 1:
                return False
            if board.board [self.x2] [self.y2] == Black:
                if self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1:
                    return True
                return False
            elif board.board [self.x2] [self.y2] == Empty:
                if self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1 or self.y2 == self.y1:
                    return True
                return False
        elif self.color == Black:
            if self.x2 != self.x1 - 1:
                return False
            if board.board [self.x2] [self.y2] == White:
                if self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1:
                    return True
                return False
            elif board.board [self.x2] [self.y2] == Empty:
                if self.y2 == self.y1 + 1 or self.y2 == self.y1 - 1 or self.y2 == self.y1:
                    return True
                return False
        return False
    
    def code (self, board):
        direction = 0
        if self.y2 > self.y1:
            if board.board [self.x2] [self.y2] == Empty:
                direction = 1
            else: 
                direction = 2
        if self.y2 < self.y1:
            if board.board [self.x2] [self.y2] == Empty:
                direction = 3
            else:
                direction = 4
        if self.color == White:
            return 5 * (Dy * self.x1 + self.y1) + direction
        else:
            return 5 * Dx * Dy + 5 * (Dy * self.x1 + self.y1) + direction


        

# Board class to play Breakthrough 5x5

In [ ]:
import random

Dx = 5
Dy = 5
Empty = 0
White = 1
Black = 2

class Board(object):
    def __init__(self):
        self.h = 0
        self.turn = White
        self.board = np.zeros ((Dx, Dy))
        for i in range (0, 2):
            for j in range (0, Dy):
                self.board [i] [j] = White
        for i in range (Dx - 2, Dx):
            for j in range (0, Dy):
                self.board [i] [j] = Black
    
    def legalMoves(self):
        moves = []
        for i in range (0, Dx):
            for j in range (0, Dy):
                if self.board [i] [j] == self.turn:
                    for k in [-1, 0, 1]:
                        for l in [-1, 0, 1]:
                            m = Move (self.turn, i, j, i + k, j + l)
                            if m.valid (self):
                                moves.append (m)
        return moves
    
    def score (self):
        for i in range (0, Dy):
            if (self.board [Dx - 1] [i] == White):
                return 1.0
            elif (self.board [0] [i] == Black):
                return 0.0
        l = self.legalMoves ()
        if len (l) == 0:
            if self.turn == Black:
                return 1.0
            else:
                return 0.0
        return 0.5

    def terminal (self):
        if self.score () == 0.5:
            return False
        return True
    
    def play (self, move):
        self.board [move.x1] [move.y1] = Empty
        self.board [move.x2] [move.y2] = move.color
        if (self.turn == White):
            self.turn = Black
        else:
            self.turn = White

    def playout (self):
        while (True):
            moves = self.legalMoves ()
            if self.terminal ():
                return self.score ()
            n = random.randint (0, len (moves) - 1)
            self.play (moves [n])

    def print(self):
        print("   1 2 3 4 5")
        for i in range(Dy):
            print("{} |".format(i + 1), end="")
            for j in range(Dx):
                if self.board [i] [j] == Black:
                    print("\u265F", end="")
                elif self.board [i] [j] == White:
                    print("\u2659", end="")
                else:
                    print(" ", end="")
                if j < Dx:
                    print("|", end="")

            if i < Dy:
                print()        


# Flat Monte Carlo

In [ ]:
import copy
def flat (board, n):
    moves = board.legalMoves ()
    bestScore = 0
    bestMove = 0
    for m in range (len(moves)):
        sum = 0
        for i in range (n // len (moves)):
            b = copy.deepcopy (board)
            b.play (moves [m])
            r = b.playout ()
            if board.turn == Black:
                r = 1 - r
            sum = sum + r
        if sum > bestScore:
            bestScore = sum
            bestMove = m
    return moves [bestMove]

# UCB

In [ ]:
def UCB (board, n):
    moves = board.legalMoves ()
    sumScores = [0.0 for x in range (len (moves))]
    nbVisits = [0 for x in range (len(moves))]
    for i in range (n):
        bestScore = 0
        bestMove = 0
        for m in range (len(moves)):
            score = 1000000
            if nbVisits [m] > 0:
                 score = sumScores [m] / nbVisits [m] + 0.4 * math.sqrt (math.log (i) / nbVisits [m])
            if score > bestScore:
                bestScore = score
                bestMove = m
        b = copy.deepcopy (board)
        b.play (moves [bestMove])
        r = b.playout ()
        if board.turn == Black:
            r = 1.0 - r
        sumScores [bestMove] += r
        nbVisits [bestMove] += 1
    bestNbVisits = 0
    bestMove = 0
    for m in range (len(moves)):
        if nbVisits [m] > bestNbVisits:
            bestNbVisits = nbVisits [m]
            bestMove = m
    return moves [bestMove]

# Board class with hashcode

In [ ]:
import random

Dx = 5
Dy = 5
Empty = 0
White = 1
Black = 2

hashTable = []
for k in range (3):
    l = []
    for i in range (Dx):
        l1 = []
        for j in range (Dy):
            l1.append (random.randint (0, 2 ** 64))
        l.append (l1)
    hashTable.append (l)
hashTurn = random.randint (0, 2 ** 64)

class Board(object):
    def __init__(self):
        self.h = 0
        self.turn = White
        self.board = np.zeros ((Dx, Dy))
        for i in range (0, 2):
            for j in range (0, Dy):
                self.board [i] [j] = White
        for i in range (Dx - 2, Dx):
            for j in range (0, Dy):
                self.board [i] [j] = Black
    
    def legalMoves(self):
        moves = []
        for i in range (0, Dx):
            for j in range (0, Dy):
                if self.board [i] [j] == self.turn:
                    for k in [-1, 0, 1]:
                        for l in [-1, 0, 1]:
                            m = Move (self.turn, i, j, i + k, j + l)
                            if m.valid (self):
                                moves.append (m)
        return moves
    
    def score(self):
        for i in range (0, Dy):
            if (self.board [Dx - 1] [i] == White):
                return 1.0
            elif (self.board [0] [i] == Black):
                return 0.0
        l = self.legalMoves ()
        if len (l) == 0:
            if self.turn == Black:
                return 1.0
            else:
                return 0.0
        return 0.5

    def terminal(self):
        if self.score () == 0.5:
            return False
        return True
    
    def playout(self):
        while (True):
            moves = self.legalMoves ()
            if self.terminal ():
                return self.score ()
            n = random.randint (0, len (moves) - 1)
            self.play (moves [n])
            
    def play(self, move):
        col = int (self.board [move.x2] [move.y2])
        if col != Empty:
            self.h = self.h ^ hashTable [col] [move.x2] [move.y2]
        self.h = self.h ^ hashTable [move.color] [move.x2] [move.y2]
        self.h = self.h ^ hashTable [move.color] [move.x1] [move.y1]
        self.h = self.h ^ hashTurn
        self.board [move.x2] [move.y2] = move.color
        self.board [move.x1] [move.y1] = Empty
        if (move.color == White):
            self.turn = Black
        else:
            self.turn = White
            
    def print(self):
        print("   1 2 3 4 5")
        for i in range(Dy):
            print("{} |".format(i + 1), end="")
            for j in range(Dx):
                if self.board [i] [j] == Black:
                    print("\u265F", end="")
                elif self.board [i] [j] == White:
                    print("\u2659", end="")
                else:
                    print(" ", end="")
                if j < Dx:
                    print("|", end="")

            if i < Dy:
                print()


# Transposition Table

In [ ]:
MaxLegalMoves = 6 * Dx
Table = {}

def add (board):
    nplayouts = [0.0 for x in range (MaxLegalMoves)]
    nwins = [0.0 for x in range (MaxLegalMoves)]
    Table [board.h] = [0, nplayouts, nwins]

def look (board):
    return Table.get (board.h, None)


# UCT

In [ ]:
def UCT (board):
    if board.terminal ():
        return board.score ()
    t = look (board)
    if t != None:
        bestValue = 0
        best = 0
        moves = board.legalMoves ()
        for i in range (0, len (moves)):
            val = 1000000.0
            n = t [0]
            ni = t [1] [i]
            wi = t [2] [i]
            if ni > 0:
                Q = wi / ni
                if board.turn == Black:
                    Q = 1 - Q
                val = Q + 0.4 * sqrt (log (n) / ni)
            if val > bestValue:
                bestValue = val
                best = i
        board.play (moves [best])
        res = UCT (board)
        t [0] += 1
        t [1] [best] += 1
        t [2] [best] += res
        return res
    else:
        add (board)
        return board.playout ()

In [ ]:
def BestMoveUCT (board, n):
    global Table
    Table = {}
    for i in range (n):
        b1 = copy.deepcopy (board)
        res = UCT (b1)
    t = look (board)
    moves = board.legalMoves ()
    best = moves [0]
    bestValue = t [1] [0]
    for i in range (1, len(moves)):
        if (t [1] [i] > bestValue):
            bestValue = t [1] [i]
            best = moves [i]
    return best

# Tournament between Flat and UCT

In [ ]:
def game ():
    b = Board ()
    while not b.terminal ():
        m = flat (b, 1000)
        b.play (m)
        #b.print ()
        if not b.terminal ():
            m = BestMoveUCT (b, 1000)
            b.play (m)
            #b.print ()
    b.print ()
    return b.score ()

s = 0
for i in range (10):
    s += game ()
    print (s)
        

# RAVE
 

In [ ]:
def playoutAMAF (board, played):
    while (True):
        moves = board.legalMoves ()
        if len (moves) == 0 or board.terminal ():
            return board.score ()
        n = random.randint (0, len (moves) - 1)
        played.append (moves [n].code (board))
        board.play (moves [n])

MaxCodeLegalMoves = 2 * Dx * Dy * 5

def addAMAF (board):
    nplayouts = [0.0 for x in range (MaxLegalMoves)]
    nwins = [0.0 for x in range (MaxLegalMoves)]
    nplayoutsAMAF = [0.0 for x in range (MaxCodeLegalMoves)]
    nwinsAMAF = [0.0 for x in range (MaxCodeLegalMoves)]
    Table [board.h] = [0, nplayouts, nwins, nplayoutsAMAF, nwinsAMAF]

def updateAMAF (t, played, res):
    for i in range (len (played)):
        if played [:i].count (played [i]) == 0:
            t [3] [played [i]] += 1
            t [4] [played [i]] += res


In [ ]:
def RAVE (board, played):
    if (board.terminal ()):
        return board.score ()
    t = look (board)
    if t != None:
        bestValue = 0
        best = 0
        moves = board.legalMoves ()
        bestcode = moves [0].code (board)
        for i in range (0, len (moves)):
            val = 1000000.0
            code = moves [i].code (board)
            if t [3] [code] > 0:
                beta = t [3] [code] / (t [1] [i] + t [3] [code] + 1e-5 * t [1] [i] * t [3] [code])
                Q = 1
                if t [1] [i] > 0:
                    Q = t [2] [i] / t [1] [i]
                    if board.turn == Black:
                        Q = 1 - Q
                AMAF = t [4] [code] / t [3] [code]
                if board.turn == Black:
                    AMAF = 1 - AMAF
                val = (1.0 - beta) * Q + beta * AMAF
            if val > bestValue:
                bestValue = val
                best = i
                bestcode = code
        board.play (moves [best])
        played.append (bestcode)
        res = RAVE (board, played)
        t [0] += 1
        t [1] [best] += 1
        t [2] [best] += res
        updateAMAF (t, played, res)
        return res
    else:
        addAMAF (board)
        return playoutAMAF (board, played)

In [ ]:
def BestMoveRAVE (board, n):
    global Table
    Table = {}
    for i in range (n):
        b1 = copy.deepcopy (board)
        res = RAVE (b1, [])
    t = look (board)
    moves = board.legalMoves ()
    best = moves [0]
    bestValue = t [1] [0]
    for i in range (1, len(moves)):
        if (t [1] [i] > bestValue):
            bestValue = t [1] [i]
            best = moves [i]
    return best

In [ ]:
def gameRAVE ():
    b = Board ()
    while not b.terminal ():
        m = flat (b, 1000)
        b.play (m)
        #b.print ()
        if not b.terminal ():
            m = BestMoveRAVE (b, 1000)
            b.play (m)
            #b.print ()
    b.print ()
    return b.score ()

s = 0
for i in range (10):
    s += gameRAVE ()
    print (s)

# GRAVE

In [ ]:
def GRAVE (board, played, tref):
    if (board.terminal ()):
        return board.score ()
    t = look (board)
    if t != None:
        tr = tref
        if t [0] > 50:
            tr = t
        bestValue = 0
        best = 0
        moves = board.legalMoves ()
        bestcode = moves [0].code (board)
        for i in range (0, len (moves)):
            val = 1000000.0
            code = moves [i].code (board)
            if tr [3] [code] > 0:
                beta = tr [3] [code] / (t [1] [i] + tr [3] [code] + 1e-5 * t [1] [i] * tr [3] [code])
                Q = 1
                if t [1] [i] > 0:
                    Q = t [2] [i] / t [1] [i]
                    if board.turn == Black:
                        Q = 1 - Q

                AMAF = tr [4] [code] / tr [3] [code]
                if board.turn == Black:
                    AMAF = 1 - AMAF
                val = (1.0 - beta) * Q + beta * AMAF
            if val > bestValue:
                bestValue = val
                best = i
                bestcode = code
        board.play (moves [best])
        played.append (bestcode)
        res = GRAVE (board, played, tr)
        t [0] += 1
        t [1] [best] += 1
        t [2] [best] += res
        updateAMAF (t, played, res)
        return res
    else:
        addAMAF (board)
        return playoutAMAF (board, played)

In [ ]:
def BestMoveGRAVE (board, n):
    global Table
    Table = {}
    addAMAF (board)
    for i in range (n):
        root = look (board)
        b1 = copy.deepcopy (board)
        res = GRAVE (b1, [], root)
    root = look (board)
    moves = board.legalMoves ()
    best = moves [0]
    bestValue = root [1] [0]
    for i in range (1, len(moves)):
        if (root [1] [i] > bestValue):
            bestValue = root [1] [i]
            best = moves [i]
    return best

In [ ]:
def gameGRAVE ():
    b = Board ()
    while not b.terminal ():
        m = flat (b, 1000)
        b.play (m)
        #b.print ()
        if not b.terminal ():
            m = BestMoveGRAVE (b, 1000)
            b.play (m)
            #b.print ()
    b.print ()
    return b.score ()

s = 0
for i in range (10):
    s += gameGRAVE ()
    print (s)